# **Computer Vision**

Import necessary libraries 

In [34]:
import os
import numpy as np
import shutil
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD

from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Dropout
from keras.regularizers import l2

**Data Collection**

In [20]:
%cd /content/drive/MyDrive/ML2_Project/data


/content/drive/MyDrive/ML2_Project/data


Extract the dataset-resized ZIP file

In [ ]:
zip_path = '/content/drive/MyDrive/ML2_Project/data/dataset-resized.zip'
destination_folder = '/content/drive/MyDrive/ML2_Project/data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)


Run resize.py to resizing images in different categories and saving them

In [ ]:
!python resize.py

**Data Preperation**

Define image data generator for training and validation data

In [21]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

Load dataset

In [22]:
dataset_dir = '/content/drive/MyDrive/ML2_Project/data/dataset-resized'

Get the list of subdirectories (classes) in the dataset directory

In [23]:
classes = sorted(os.listdir(dataset_dir))

Create the list of directories for train and test sets

In [24]:
train_dir = '/content/drive/MyDrive/ML2_Project/data/train'
test_dir = '/content/drive/MyDrive/ML2_Project/data/test'

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

Data split and directory creation

In [15]:
# Split the dataset into training and test sets for each class
for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    
    # Skip if it's not a directory
    if not os.path.isdir(class_dir):
        continue
    
    files = [file for file in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, file))]
    labels = [class_name] * len(files)  # Assign the class label to each file
    
    train_files, test_files, train_labels, test_labels = train_test_split(files, labels, test_size=0.2, random_state=42, stratify=labels)

    # Create subdirectories for each class in train and test directories
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    # Move train files to train class directory
    for file in train_files:
        src = os.path.join(class_dir, file)
        dst = os.path.join(train_class_dir, file)
        shutil.copyfile(src, dst)
        
    # Move test files to test class directory
    for file in test_files:
        src = os.path.join(class_dir, file)
        dst = os.path.join(test_class_dir, file)
        shutil.copyfile(src, dst)

Data Generators

Create the generators for training and testing

In [25]:
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 2216 images belonging to 6 classes.
Found 705 images belonging to 6 classes.


**Modeling**

Build Model

In [26]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))  
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))  
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))  
model.add(Flatten())
model.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(units=6, activation='softmax'))

Compile Model

In [27]:
# optimizer = Adam(learning_rate=0.001)
# optimizer = SGD(learning_rate=0.001)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])

Train Model

In [29]:
history = model.fit(train_set, epochs=25, batch_size=64, validation_data=test_set)

Epoch 1/25
70/70 [==============================] - 85s 1s/step - loss: 2.2503 - accuracy: 0.4540 - val_loss: 3.2678 - val_accuracy: 0.1560
Epoch 2/25
70/70 [==============================] - 42s 597ms/step - loss: 1.8835 - accuracy: 0.5361 - val_loss: 3.4725 - val_accuracy: 0.1901
Epoch 3/25
70/70 [==============================] - 41s 589ms/step - loss: 1.7418 - accuracy: 0.5862 - val_loss: 3.3589 - val_accuracy: 0.2071
Epoch 4/25
70/70 [==============================] - 41s 589ms/step - loss: 1.6039 - accuracy: 0.6331 - val_loss: 3.2703 - val_accuracy: 0.1957
Epoch 5/25
70/70 [==============================] - 42s 587ms/step - loss: 1.5802 - accuracy: 0.6327 - val_loss: 2.5697 - val_accuracy: 0.2837
Epoch 6/25
70/70 [==============================] - 41s 585ms/step - loss: 1.4482 - accuracy: 0.6837 - val_loss: 3.0177 - val_accuracy: 0.2624
Epoch 7/25
70/70 [==============================] - 42s 590ms/step - loss: 1.3530 - accuracy: 0.7071 - val_loss: 2.6125 - val_accuracy: 0.3333
Ep

**Evaluation**

Evaluate the model on the test data

In [30]:
test_loss, test_accuracy = model.evaluate(test_set)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

23/23 [==============================] - 7s 263ms/step - loss: 1.7803 - accuracy: 0.6028
Test Loss: 1.780347228050232
Test Accuracy: 0.6028369069099426


Get prediction and true labels for the test set

In [35]:
# Get predictions for the test set
y_pred = model.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)  # Convert one-hot encoded predictions to class indices

# Get the true labels for the test set
y_true = test_set.classes

23/23 [==============================] - 5s 225ms/step


Create confusion matrix

In [36]:
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[19 14 21 37 15  8]
 [18 18 22 56 17 10]
 [21 17 12 39 15  7]
 [28 17 14 63 18 23]
 [25 10 17 52 20 13]
 [10  3  4 15  1  6]]


Get classification report

In [37]:
class_labels = test_set.class_indices
class_names = list(class_labels.keys())
report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.16      0.17      0.16       114
       glass       0.23      0.13      0.16       141
       metal       0.13      0.11      0.12       111
       paper       0.24      0.39      0.30       163
     plastic       0.23      0.15      0.18       137
       trash       0.09      0.15      0.11        39

    accuracy                           0.20       705
   macro avg       0.18      0.18      0.17       705
weighted avg       0.20      0.20      0.19       705



**Application**

Test 1

In [38]:
from PIL import Image
import numpy as np

# Load and preprocess the test image
test_image_path = '/content/drive/MyDrive/ML2_Project/data/test/glass/glass110.jpg'
test_image = Image.open(test_image_path)
target_size = (64, 64)
test_image = test_image.resize(target_size)
test_image = np.array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Make predictions on the test image
predictions = model.predict(test_image)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map the predicted class index to the corresponding class label
class_labels = train_set.class_indices
predicted_class_label = {v: k for k, v in class_labels.items()}[predicted_class_index]

# Print the predicted class label
print('Predicted class:', predicted_class_label)


1/1 [==============================] - 0s 201ms/step
Predicted class: glass


Test 2

In [39]:
from PIL import Image
import numpy as np

# Load and preprocess the test image
test_image_path = '/content/drive/MyDrive/ML2_Project/data/test/plastic/plastic10.jpg'
test_image = Image.open(test_image_path)
target_size = (64, 64)
test_image = test_image.resize(target_size)
test_image = np.array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Make predictions on the test image
predictions = model.predict(test_image)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map the predicted class index to the corresponding class label
class_labels = train_set.class_indices
predicted_class_label = {v: k for k, v in class_labels.items()}[predicted_class_index]

# Print the predicted class label
print('Predicted class:', predicted_class_label)

1/1 [==============================] - 0s 27ms/step
Predicted class: plastic


Save and Download Model (So that we can use it in the NLP part)

In [ ]:
model.save('/content/drive/MyDrive/ML2_Project/model/trash_classifier.h5')